![email inbox](email_inbox.jpg)

Every day, professionals wade through hundreds of emails, from urgent client requests to promotional offers. It's like trying to find important messages in a digital ocean. But AI can help you stay afloat by automatically sorting emails to highlight what matters most.

You've been asked to build an intelligent email assistant using Llama, to help users automatically classify their incoming emails. Your system will identify which emails need immediate attention, which are regular updates, and which are promotions that can wait or be archived.

### The Data
You'll work with a dataset of various email examples, ranging from urgent business communications to promotional offers. Here's a peek at what you'll be working with:

### email_categories_data.csv

 Column | Description |
|--------|-------------|
| email_id | A unique identifier for each email in the dataset. |
| email_content | The full email text including subject line and body. Each email follows a format of "Subject" followed by the message content on a new line. |
| expected_category | The correct classification of the email: `Priority`, `Updates`, or `Promotions`. This will be used to validate your model's performance. |



In [24]:
# Run the following cells first
# Install necessary packages, then import the model running the cell below
!pip install llama-cpp-python==0.2.82 -q -q -q

In [25]:
SELECT *
FROM 'models.csv'
LIMIT 5

column0  ...                                            column2
0                            model  ...                                             source
1  tinyllama-1.1b-chat-v0.3.Q4_K_M  ...  https://huggingface.co/TheBloke/TinyLlama-1.1B...
2       llama-3.2-3b-instruct-q8_0  ...  https://huggingface.co/hugging-quants/Llama-3....

[3 rows x 3 columns]

In [26]:
# Import required libraries
import pandas as pd
from llama_cpp import Llama

In [27]:
# Load the email dataset
emails_df = pd.read_csv('data/email_categories_data.csv')
# Display the first few rows of our dataset
print("Preview of our email dataset:")
emails_df.head(2)

Preview of our email dataset:


email_id  ... expected_category
0         1  ...          Priority
1         2  ...        Promotions

[2 rows x 3 columns]

In [28]:
# Set the model path
model_path = "/files-integrations/files/c9696c24-44f3-45f7-8ccd-4b9b046e7e53/tinyllama-1.1b-chat-v0.3.Q4_K_M.gguf"

In [29]:
llm = Llama(model_path=model_path)

prompt = """Classify the emails into three categories: Priority, Updates, Promotions

Example 1:
Urgent: Password Reset Required
Your account security requires immediate attention. Please reset your password within 24 hours.
Response: Priority

Example 2:
Special Offer - 50 Off Everything!
Don't miss our biggest sale of the year!
Response: Promotions

Example 3:
Canceled Event - Team Meeting
This event has been canceled and removed from your calendar.
Response: Updates

Example 4:

"""

llama_model_loader: loaded meta data with 20 key-value pairs and 201 tensors from /files-integrations/files/c9696c24-44f3-45f7-8ccd-4b9b046e7e53/tinyllama-1.1b-chat-v0.3.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = py007_tinyllama-1.1b-chat-v0.3
llama_model_loader: - kv   2:                       llama.context_length u32              = 2048
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 2048
llama_model_loader: - kv   4:                          llama.block_count u32              = 22
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 5632
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 64


In [30]:
def process_message(llm, message, prompt):
    """Process a message and return the response"""
    input = f"{prompt}{message}"
    response = llm(
        input,
        max_tokens=20,
        temperature=0,
        stop=["Q:","\n"]
    )
    return response['choices'][0]['text'].strip()

In [33]:
test_emails = emails_df.head(2)

# Process each test email and store results
results = []
for idx, row in test_emails.iterrows():
    email_content = row['email_content']
    expected_category = row['expected_category']
    
    # Get model's classification
    result = process_message(llm, email_content, prompt)
    
    # Store results
    results.append({
        'email_content': email_content,
        'expected_category': expected_category,
        'model_output': result
    })

results_df = pd.DataFrame(results)
results_df

# Print the results based on the name of the output
for idx, row in results_df.iterrows():
    print(f"Email Content: {row['email_content']}")
    print(f"Expected Category: {row['expected_category']}")
    print(f"Model Output: {row['model_output']}")
    print("\n")

Llama.generate: prefix-match hit

llama_print_timings:        load time =   35024.97 ms
llama_print_timings:      sample time =       0.20 ms /     1 runs   (    0.20 ms per token,  4975.12 tokens per second)
llama_print_timings: prompt eval time =   28002.01 ms /    29 tokens (  965.59 ms per token,     1.04 tokens per second)
llama_print_timings:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       total time =   28003.07 ms /    30 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =   35024.97 ms
llama_print_timings:      sample time =       0.17 ms /     1 runs   (    0.17 ms per token,  6060.61 tokens per second)
llama_print_timings: prompt eval time =   27207.17 ms /    29 tokens (  938.18 ms per token,     1.07 tokens per second)
llama_print_timings:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_print_timings:       to

Email Content: Urgent: Server Maintenance Required\nOur main server needs immediate maintenance due to critical errors. Please address ASAP.
Expected Category: Priority
Model Output: 


Email Content: 50% Off Spring Collection!\nDon't miss our biggest sale of the season! All spring items half off. Limited time offer.
Expected Category: Promotions
Model Output: 


